# Grabbing replay logs from Pokemon Showdown

In [1]:
import requests
from tqdm import tqdm
import pandas as pd
import re

In [20]:
pkmon_format = 'gen8vgc2021series11'
showdown_replay_url = f'https://replay.pokemonshowdown.com/search.json?format={pkmon_format}'

def get_replay_id(replay_url: str, limit: int) -> list[str]:
    replay_id = []
    for i in range(limit):
        data = requests.get(f'{replay_url}&page={i+1}').json()
        for item in data:
            replay_id.append(item['id'])
    return replay_id

def get_replay_data(replay_id: str) -> dict:
    game_log_url = f'https://replay.pokemonshowdown.com/{replay_id}.json'
    return requests.get(game_log_url).json()

def get_data(replay_url:str) -> list[dict]:
    replay_ids = get_replay_id(replay_url, 25)
    return [get_replay_data(x) for x in tqdm(replay_ids)]

data = get_data(showdown_replay_url)

# Saving data to CSV for easier access

In [22]:
df = pd.DataFrame(data)
df.to_csv("data.csv", sep=',',index=False)

# pandas.read_csv("data.csv") to import data again

# Loading data

In [91]:
data = pd.read_csv('data.csv')

log = list(data['log'])

# Finds pokemon in each team

In [92]:
def get_teams(log): 
    team_dict = {}
    for item in log.split('\n'):
        if '|poke|' in item:
            team = re.search('p[1-2]', item).group()
            pokemon = item.split('|')[-2].split(',')[0]
            if team in team_dict:
                team_dict[team].append(pokemon)
            else:
                team_dict[team] = [pokemon]
    return team_dict

get_teams(log[0])

{'p1': ['Zacian-*',
  'Incineroar',
  'Torkoal',
  'Dracovish',
  'Venusaur',
  'Landorus-Therian'],
 'p2': ['Corviknight',
  'Charizard',
  'Grimmsnarl',
  'Venusaur',
  'Snorlax',
  'Blastoise']}

# Splits up replay parts into turns

In [93]:
temp = re.search('\|start.+\|win',re.sub('\n', '-n-', log[0])).group()
turns = [re.sub('-n-', '\n', x) for x in temp.split('|turn|')]
for t in turns:
    print(t)

|start
|switch|p1a: Dracovish|Dracovish, L50|100/100
|switch|p1b: Incineroar|Incineroar, L50, M|100/100
|switch|p2a: Charizard|Charizard, L50, M|100/100
|switch|p2b: Grimmsnarl|Grimmsnarl, L50, M|100/100
|-ability|p1b: Incineroar|Intimidate|boost
|-unboost|p2a: Charizard|atk|1
|-unboost|p2b: Grimmsnarl|atk|1

1
|
|t:|1638041276
|-start|p1a: Dracovish|Dynamax
|-heal|p1a: Dracovish|100/100|[silent]
|-start|p2b: Grimmsnarl|Dynamax
|-formechange|p2b: Grimmsnarl|Grimmsnarl-Gmax
|-heal|p2b: Grimmsnarl|100/100|[silent]
|move|p1b: Incineroar|Fake Out|p2a: Charizard
|-damage|p2a: Charizard|86/100
|move|p1a: Dracovish|Max Rockfall|p2a: Charizard
|-supereffective|p2a: Charizard
|-damage|p2a: Charizard|0 fnt
|-weather|Sandstorm
|faint|p2a: Charizard
|move|p2b: Grimmsnarl|Max Starfall|p1a: Dracovish
|-supereffective|p1a: Dracovish
|-damage|p1a: Dracovish|40/100
|-fieldstart|move: Misty Terrain
|
|-weather|Sandstorm|[upkeep]
|-damage|p1a: Dracovish|37/100|[from] Sandstorm
|-damage|p2b: Grimmsnarl|97

In [118]:
# temp = turns[1].split('\n')
'''
Find the 'keys' -> headers that indicate action during the turn
'''
def count_keys(turns):
    counter = {}
    for turn in turns:
        temp = turn.split('\n')
        for item in temp:
            if item[:2] == '|-':
                key = re.search('^\|-\S+\|',item).group()
                item = re.sub(key, '', item)
                if key in counter:
                    # counter[key] += [item]
                    counter[key] += 1
                else:
                    # counter[key] = [item]
                    counter[key] = 1
    return counter

count_keys(turns)

{'|-ability|': 2,
 '|-boost|': 1,
 '|-start|': 2,
 '|-heal|': 3,
 '|-unboost|': 9,
 '|-sidestart|': 1,
 '|-supereffective|': 9,
 '|-enditem|': 1,
 '|-damage|': 22,
 '|-resisted|': 4,
 '|-weather|': 1,
 '|-weather|Hail|': 3,
 '|-singleturn|': 1,
 '|-activate|': 1,
 '|-crit|': 3,
 '|-hitcount|': 1,
 '|-end|': 1,
 '|-sideend|': 1}

In [117]:
'''
Find info for the first turn
'''

'''
Find out which pokemon trainers initially sent out and what abilities got activated
'''
def starting_turn_actions(turn):
    starting_pkmon = []
    abilities = []
    for item in turn:
        if '|switch|' in item:
            starter = re.search('p[1-2][a-b]:.+\|', item).group().split('|')[0].split()[1]
            starting_pkmon.append(starter)
        elif '|-ability|' in item:
            ability = re.search('p[1-2][a-b]:.+', item).group().split('|')[1]
            abilities.append(re.sub('\s', '', ability))
    return {
        'starting_pkmon': starting_pkmon,
        'abilities': abilities
    }



temp = re.search('\|start.+\|win',re.sub('\n', '-n-', log[950])).group()
turns = [re.sub('-n-', '\n', x) for x in temp.split('|turn|')]
starting_turn_actions(turns[0].split('\n'))

{'starting_pkmon': ['Whimsicott', 'Dialga', 'Tornadus', 'Zamazenta'],
 'abilities': ['DauntlessShield', 'Pressure']}